In [1]:
import argparse
import glob
from hydra import compose, initialize
import hydra
import numpy as np
import pandas as pd
import torch
import yaml
from omegaconf import DictConfig, OmegaConf
from dl4cv.utils.utils import set_seed
from dl4cv.lightning_classes.plmodel import LitODModel
from dl4cv.datasets import build_taco 
from dl4cv.datasets.taco_data import taco_val_test_collate_fn
from dl4cv.utils.object_detect_utils import get_iou, fix_orientation
from tqdm import tqdm
import os


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="./configs")
inference_cfg = compose(config_name="config_taco_training")
inference_cfg["inference"]["run_name"] = 'Resnet50_Whale_bs1'
inference_cfg["inference"]["device"] = 'cuda:1'
path = f"outputs/{inference_cfg.inference.run_name}/.hydra/config.yaml"
with open(path) as cfg:
    cfg_yaml = yaml.safe_load(cfg)
cfg_yaml["inference"] = inference_cfg["inference"]
cfg_yaml["datamodule"]["test"]["params"]["num_to_return"] = 100
cfg = OmegaConf.create(cfg_yaml)


In [3]:
def main(cfg: DictConfig) -> None:
    """
    Run pytorch-lightning model inference
    Args:
        cfg: hydra config
    Returns:
        None
    """
    set_seed(cfg.training.seed)

    device = torch.device(cfg.inference.device)

    model_names = glob.glob(f"outputs/{cfg.inference.run_name}/saved_models/*.ckpt")
    _, _, test_set,_ = build_taco(cfg)
    test_set.num_to_return = 100
    # Dirty trick to get the ground truth boxes
    loader = torch.utils.data.DataLoader(
        test_set,
        collate_fn=taco_val_test_collate_fn,
        batch_size=1,
        num_workers=1,
        shuffle=False,
    )
    lit_model = LitODModel.load_from_checkpoint(checkpoint_path=model_names[0], cfg=cfg)
    lit_model.to(device)
    predictions = []
    for batch in tqdm(loader):
        # move batch elements to device
        batch = tuple(b.to(device) for b in batch)
        predictions.append(
            lit_model.nms_on_image(
                batch,
            )
        )

    return predictions




In [4]:
predictions = main(cfg)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [02:50<00:00,  1.47it/s]


In [10]:
predictions[60]

In [11]:
# find an index in predictions where the kept_preds are not empty
for i in range(len(predictions)):
    if len(predictions[i]['kept_preds']) > 0:
        print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
15
16
17
18
19
20
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
56
57
59
60
62
63
64
65
66
67
68
69
70
71
72
73
74
75
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
196
197
198
199
200
201
202
203
204
205
207
209
210
211
212
213
214
215
216
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [7]:
predictions[225]

{'kept_preds': [{'bbox': {'x1': 319, 'y1': 566, 'x2': 783, 'y2': 705},
   'conf': 0.9992782473564148,
   'pred_class': 21,
   'true_class': 21,
   'image_id': 378},
  {'bbox': {'x1': 365, 'y1': 56, 'x2': 877, 'y2': 200},
   'conf': 0.999017596244812,
   'pred_class': 21,
   'true_class': 21,
   'image_id': 378},
  {'bbox': {'x1': 760, 'y1': 82, 'x2': 1021, 'y2': 192},
   'conf': 0.5796942114830017,
   'pred_class': 27,
   'true_class': 28,
   'image_id': 378}],
 'P': [{'bbox': {'x1': 335, 'y1': 559, 'x2': 777, 'y2': 699},
   'conf': 0.9426859617233276,
   'pred_class': 21,
   'true_class': 21,
   'image_id': 378},
  {'bbox': {'x1': 321, 'y1': 560, 'x2': 777, 'y2': 704},
   'conf': 0.9988164901733398,
   'pred_class': 21,
   'true_class': 21,
   'image_id': 378},
  {'bbox': {'x1': 322, 'y1': 569, 'x2': 776, 'y2': 700},
   'conf': 0.9816123843193054,
   'pred_class': 21,
   'true_class': 21,
   'image_id': 378},
  {'bbox': {'x1': 322, 'y1': 566, 'x2': 776, 'y2': 701},
   'conf': 0.992823

In [8]:
def mean_average_precision(pred, truth, iou_threshold = 0.5, num_classes = 29, per_class = False):
    # compute mAP https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/object_detection/metrics/mean_avg_precision.py
    # `pred` is given in [[{'bbox':{'x1', 'x2', 'y1', 'y2'}, 'class'(int), 'conf'}, ...], ...]
    # `truth` is given in [[{'x1', 'x2', 'y1', 'y2', 'class'(int)}, more boxes...], ...]
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(1, num_classes): # class '0' is background

        TP = 0
        FP = 0
        total_true_bboxes = 0

        # list detected(predicted) objects of class 'c'
        detections = []

        for idx, prs in enumerate(pred):
          for pr in prs:
            if pr['class'] == c:
                detections.append((pr['conf'], idx, pr['bbox']))

        # make checkbox for checking whether gt object was detected
        total_true_bboxes = 0
        is_detected = []
        for gts in pred:
          is_detected.append([False for _ in gts])
          total_true_bboxes += sum([gt['class']==c for gt in gts])

        detections.sort(reverse=True)

        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))

        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            num_gts = len(truth[detection[1]])

            # find most closest g.t box to pred as best_gt_idx
            best_iou = 0
            for idx, gt in enumerate(truth[detection[1]]):
                #print(gt, detection[2])
                
                iou = get_iou(gt, detection[2])

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            # if considered found
            #try:
            #  print(best_iou, truth[detection[1]][best_gt_idx], detection[2])
            #except:
            #  pass
            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if is_detected[detection[1]][best_gt_idx] == False:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    is_detected[detection[1]][best_gt_idx] = True
                else: # duplicate is FP
                    FP[detection_idx] = 1
            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1
        
        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        #if len(TP)>0 and len(FP)>0:
        #  print(TP_cumsum[-1], FP_cumsum[-1])
        #print(total_true_bboxes)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)  # ratio of detected objects!
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)         # ratio of predictions that are true objects!

        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        #print(precisions, recalls, torch.trapz(precisions, recalls))
        average_precisions.append(torch.trapz(precisions, recalls))
        #print('----------')
    if per_class: 
        return average_precisions
    else:
        return sum(average_precisions) / len(average_precisions)

In [9]:
predictions[92]

{'kept_preds': [{'bbox': {'x1': 150, 'y1': 155, 'x2': 1152, 'y2': 444},
   'conf': 0.9996634721755981,
   'pred_class': 9,
   'true_class': 9,
   'image_id': 823},
  {'bbox': {'x1': 150, 'y1': 238, 'x2': 1202, 'y2': 583},
   'conf': 0.9391825795173645,
   'pred_class': 9,
   'true_class': 2,
   'image_id': 823},
  {'bbox': {'x1': 170, 'y1': 181, 'x2': 564, 'y2': 430},
   'conf': 0.7873830199241638,
   'pred_class': 9,
   'true_class': 28,
   'image_id': 823},
  {'bbox': {'x1': 134, 'y1': 421, 'x2': 879, 'y2': 590},
   'conf': 0.6445581912994385,
   'pred_class': 2,
   'true_class': 28,
   'image_id': 823},
  {'bbox': {'x1': 178, 'y1': 383, 'x2': 1176, 'y2': 519},
   'conf': 0.37869709730148315,
   'pred_class': 9,
   'true_class': 28,
   'image_id': 823}],
 'P': [{'bbox': {'x1': 150, 'y1': 152, 'x2': 1054, 'y2': 431},
   'conf': 0.9085447192192078,
   'pred_class': 9,
   'true_class': 9,
   'image_id': 823},
  {'bbox': {'x1': 150, 'y1': 238, 'x2': 1202, 'y2': 583},
   'conf': 0.9391825